In [35]:
import sys; sys.path.append('methods/')
import os
from regridding import *

In [2]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [26]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.lat - x)**2 + (da.lon - y)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 0.125)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["lat", "lon"])
   
    return val

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile for the 2019 constituencies
sf = geopandas.read_file("Westminster_Parliamentary_Constituencies_December_2019_Boundaries_UK_BUC")
sf

,pcon19cd,pcon19nm,bng_e,bng_n,long,lat,objectid,st_lengths,st_areasha,geometry
0,E14000530,Aldershot,484884,155126,-0.78410,51.288952,1,37241.703656,5.258835e+07,"POLYGON ((485406.902 159918.603, 486138.297 15..."
1,E14000531,Aldridge-Brownhills,404723,302568,-1.93166,52.620869,2,35258.924699,4.398713e+07,"POLYGON ((406519.098 305054.298, 405782.501 30..."
2,E14000532,Altrincham and Sale West,374132,389051,-2.39049,53.397659,3,40812.004306,5.087472e+07,"POLYGON ((379104.096 393143.903, 377127.904 39..."
3,E14000533,Amber Valley,440478,349675,-1.39770,53.042831,4,54421.854957,1.246901e+08,"POLYGON ((444868.402 353958.100, 444113.303 35..."
4,E14000534,Arundel and South Downs,510686,115542,-0.42635,50.928711,5,207778.021149,6.469497e+08,"POLYGON ((506641.497 128757.203, 506389.800 12..."
...,...,...,...,...,...,...,...,...,...,...
645,W07000076,Caerphilly,316860,188276,-3.20144,51.587181,646,63865.409578,1.150569e+08,"POLYGON ((315131.401 200637.498, 315308.299 19..."
646,W07000077,Islwyn,320517,196276,-3.15048,51.659618,647,59239.492373,1.117145e+08,"POLYGON ((322947.397 199706.204, 322974.804 19..."
647,W07000078,Vale of Glamorgan,302293,173081,-3.40742,51.448250,648,91342.706619,3.148419e+08,"POLYGON ((302260.803 179531.999, 302783.903 17..."
648,W07000079,Cardiff West,312437,178939,-3.26294,51.502571,649,36926.700752,5.081285e+07,"POLYGON ((312599.599 182852.798, 312692.900 18..."


In [29]:
filesnames=["CORDEX-nearterm_rcp45_conseq-dry-days.nc","CORDEX-nearterm_rcp45_rx5day.nc","ERA5-trend_pr_DJF.nc","CORDEX-nearterm_rcp45_maxtmax.nc","CORDEX-nearterm_rcp45_temp_ANN.nc","ERA5-trend_pr_JJA.nc","CORDEX-nearterm_rcp45_pr_ANN.nc","ERA5-trend_conseq-dry-days.nc","ERA5-trend_rx5day.nc","CORDEX-nearterm_rcp45_pr_DJF.nc","ERA5-trend_maxtmax.nc","ERA5-trend_temp_ANN.nc","CORDEX-nearterm_rcp45_pr_JJA.nc","ERA5-trend_pr_ANN.nc"]
varnames_in=["cdd_anom","rx5day_anom","pr_trend","txx_anom","t_anom","pr_trend","pr_anom","cdd_trend","rx5day_trend","pr_anom","txx_trend","t_trend","pr_anom","pr_trend"]
varnames_out=["cdd_anom","rx5day_anom","pr_djf_trend","maxtmax_anom","t_anom","pr_jja_trend","pr_ann_anom","cdd_trend","rx5day_anom","pr_djf_anom","txx_trend","t_trend","pr_jja_anom","pr_ann_trend"]


In [31]:
for i in range(filesnames.__len__()):
    fnm = "../gridded_data/Copernicus_Atlas_data/"+filesnames[i]
    ds = xr.open_dataset(fnm)
    #ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
    da=ds[varnames_in[i]]
    #print(da)
    rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "objectid")
    #print(rm)
    # apply the regionmask to the data and average over the x & y dimensions
    region_da = da.where(rm).mean(["lat", "lon"])
    #print(region_da)
    # find nearest neighbour (this takes the most time)
    region_nn = xr.concat([nearest_px(sf.lat[i],sf.long[i],da).expand_dims(region = [sf.objectid[i]]).assign_coords(constituency = ("region", [sf.pcon19cd[i]])) for i in range(len(sf))], "region")
    print(region_nn)
    # combine regionmask with nearest neighbour where regionmask didn't pick anything up
    region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")
    region_all=region_all.assign_coords(name=('region',sf.pcon19nm))
    # output the values into a csv file
    region_all.to_dataframe(name=varnames_out[i]).reset_index().to_csv(varnames_out[i]+'.csv')
    print("outputting ",varnames_out[i]+'.csv')




<xarray.DataArray 'cdd_anom' (region: 650)>
array([ 2.42757797e-01, -1.60142899e-01, -4.22463417e-01,  7.24601746e-02,
        3.89131546e-01, -1.89130783e-01,  4.40580368e-01, -4.07248497e-01,
        5.34780502e-01,  4.48554993e-01,  2.86954880e-01, -4.42037582e-02,
       -4.42037582e-02, -2.44202614e-01,  2.15217590e-01,  4.90579605e-01,
       -2.99276352e-01,  5.13044357e-01, -9.78288651e-02,  3.63042831e-01,
        2.92755127e-01,  2.20289230e-01,  2.65939713e-01,  2.42031097e-01,
       -4.46377754e-01,  1.02178574e-01, -4.19562340e-01,  4.20289993e-01,
        3.21016312e-01, -4.18839455e-01,  4.41970825e-02, -2.73189545e-01,
        4.41970825e-02, -1.23233795e-02,  4.41970825e-02,  4.41970825e-02,
       -1.60142899e-01,  4.41970825e-02, -1.23233795e-02, -3.21743965e-01,
       -4.99999046e-01, -4.19565201e-01, -2.81162262e-01, -3.60873222e-01,
       -3.17390442e-01, -5.31884193e-01,  3.55073929e-01, -6.73904419e-02,
       -4.09420967e-01, -4.09420967e-01, -4.57245827e-01

In [38]:
# Combine into a big dataframe
import pandas
all_files = os.listdir()    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
for i in range(csv_files.__len__()):
    this_csv=pandas.read_csv(csv_files[i])
    if i == 0:
        csv_out=this_csv.iloc[:,[2,3,4]]
    else:
        var=csv_files[i].rsplit( ".", 1 )[ 0 ]
        csv_out[var]=this_csv.iloc[:,4]

csv_out.to_csv('../by_mp/rcp45_climate_oldconstituencies.csv',index=False)




In [53]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def nearest_px(x,y,da):
   
    print('x:',x,' y:',y)
    # get squared distance from (x,y) to each point
    dist2 = (da.lat - x)**2 + (da.lon - y)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 0.125)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["lat", "lon"])
   
    return val

In [54]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile for the 2024 constituencies
sf = geopandas.read_file("PCON_MAY_2024_UK_BFE")
sf

,PCON24CD,PCON24NM,PCON24NMW,BNG_E,BNG_N,LAT,LONG,GlobalID,geometry
0,E14001063,Aldershot,None,484716,155270,51.2903,-0.78648,b8b6c57e-6fe4-432c-9130-f4d6e4598d19,"POLYGON Z ((483364.601 160961.805 0.000, 48337..."
1,E14001064,Aldridge-Brownhills,None,404720,301030,52.6071,-1.93173,d3c0545a-9c68-4095-9b85-330c7141dbf2,"POLYGON Z ((406519.098 305054.298 0.000, 40648..."
2,E14001065,Altrincham and Sale West,None,374132,389051,53.3977,-2.39049,a7e18d2c-8dd0-4f1f-b158-48799d7f6259,"POLYGON Z ((377443.302 393344.296 0.000, 37745..."
3,E14001066,Amber Valley,None,440478,349674,53.0428,-1.39771,e38d2edf-19ce-4090-ab8e-c78449d3983f,"POLYGON Z ((436223.299 356984.804 0.000, 43624..."
4,E14001067,Arundel and South Downs,None,497309,118530,50.9580,-0.61585,4a4d2a3a-a7a5-4a91-a90f-0692b459c37c,"POLYGON Z ((505688.454 133874.110 0.000, 50569..."
...,...,...,...,...,...,...,...,...,...
645,W07000108,Swansea West,Gorllewin Abertawe,264670,195124,51.6386,-3.95702,ab9c216c-374a-4dda-aabb-b5d4095808c5,"POLYGON Z ((266627.504 191650.700 0.000, 26662..."
646,W07000109,Torfaen,Torfaen,327459,200480,51.6984,-3.05102,fe1b6677-bcf4-49d8-a516-85b42b10af00,"POLYGON Z ((333723.000 192653.903 0.000, 33372..."
647,W07000110,Vale of Glamorgan,Bro Morgannwg,301298,173080,51.4481,-3.42174,43710abb-9c64-41bd-9f14-6e8120252dc2,"POLYGON Z ((302260.803 179531.999 0.000, 30226..."
648,W07000111,Wrexham,Wrecsam,337298,348629,53.0313,-2.93642,95c39b06-f367-4432-9aaa-de2a48654814,"POLYGON Z ((323631.902 349914.197 0.000, 32363..."


In [59]:
#for i in range(filesnames.__len__()):
for i in range(1):
    fnm = "../gridded_data/Copernicus_Atlas_data/"+filesnames[i]
    ds = xr.open_dataset(fnm)
    #ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
    da=ds[varnames_in[i]]
    #print(da)
    rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False)
    print(rm)
    # apply the regionmask to the data and average over the x & y dimensions
    region_da = da.where(rm).mean(["lat", "lon"])
    #print(region_da)
    # find nearest neighbour (this takes the most time)
    #region_nn = xr.concat([nearest_px(sf.LAT[i],sf.LONG[i],da).expand_dims(region = [sf.GlobalID[i]]).assign_coords(constituency = ("region", [sf.PCON24CD[i]])) for i in range(len(sf))], "region")
    region_nn = xr.concat([nearest_px(sf.LAT[i],sf.LONG[i],da).expand_dims(region = [sf.GlobalID[i]]).assign_coords(constituency = ("region", [sf.PCON24CD[i]])) for i in range(len(sf))], "region")
    print(region_nn)
    # combine regionmask with nearest neighbour where regionmask didn't pick anything up
    region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")
    region_all=region_all.assign_coords(name=('region',sf.PCON24CD))
    # output the values into a csv file
    region_all.to_dataframe(name=varnames_out[i]).reset_index().to_csv(varnames_out[i]+'.csv')
    print("outputting ",varnames_out[i]+'.csv')

<xarray.DataArray 'mask' (region: 650, lat: 88, lon: 82)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, Fals

In [ ]:
all_files = os.listdir()    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
for i in range(csv_files.__len__()):
    this_csv=pandas.read_csv(csv_files[i])
    if i == 0:
        csv_out=this_csv.iloc[:,[2,3,4]]
    else:
        var=csv_files[i].rsplit( ".", 1 )[ 0 ]
        csv_out[var]=this_csv.iloc[:,4]

csv_out.to_csv('../by_mp/rcp45_climate_newconstituencies.csv',index=False)

In [60]:
region_nn

<xarray.DataArray 'cdd_anom' (region: 650)>
array([ 2.42757797e-01, -1.60142899e-01, -4.22463417e-01,  7.24601746e-02,
        3.50723267e-01, -1.89130783e-01,  5.60146332e-01, -4.07248497e-01,
        3.91304016e-01,  5.82607269e-01,  2.86954880e-01, -4.42037582e-02,
       -4.42037582e-02, -2.44202614e-01,  2.15217590e-01,  4.01445389e-01,
       -2.99276352e-01,  5.13044357e-01,  3.63042831e-01,  2.92755127e-01,
        2.42031097e-01,  2.65939713e-01,  2.42031097e-01,  1.02178574e-01,
       -1.73883438e-02,  3.26810837e-01,  3.21016312e-01,  5.21738052e-01,
       -3.42753410e-01,  4.41970825e-02, -2.73189545e-01,  4.41970825e-02,
       -1.23233795e-02,  4.41970825e-02,  4.41970825e-02, -1.60142899e-01,
        4.41970825e-02, -1.23233795e-02, -4.02173996e-01, -4.99999046e-01,
       -4.19565201e-01, -2.81162262e-01, -3.60873222e-01, -5.14491081e-01,
       -4.25360680e-01,  2.60868073e-01, -6.73904419e-02, -4.09420967e-01,
       -4.09420967e-01, -4.57245827e-01, -4.55799103e-01,  2.64492035e-01,
        7.97828674e-01,  1.11956978e+00,  2.42757797e-01,  7.25746155e-04,
        9.42058563e-02,  9.42058563e-02,  4.76087570e-01,  2.34785080e-01,
        3.29704285e-01,  5.07972717e-01,  3.57248306e-01,  7.85503387e-01,
       -6.60871506e-01, -3.85510445e-01,  3.24640274e-01,  1.50001526e-01,
        5.61592102e-01,  5.61592102e-01,  5.61592102e-01,  2.97103882e-01,
        5.61592102e-01,  2.92751312e-01,  4.89858627e-01, -7.24601746e-02,
        2.42029190e-01, -5.72452545e-02,  2.46377945e-01, -2.31157303e-01,
...
        8.69655609e-03, -2.61594772e-01, -3.06519508e-01, -3.62318039e-01,
       -2.87680626e-01, -2.62316704e-01, -2.05071449e-01,  1.26814842e-01,
       -2.21013069e-01, -1.84782028e-01, -4.15216446e-01, -3.65942955e-01,
       -4.35505867e-01, -5.77535629e-01, -5.77535629e-01, -3.63766670e-01,
       -3.63766670e-01, -2.77535439e-01, -3.19562912e-01, -1.24639511e-01,
       -2.71739006e-01, -3.19562912e-01, -2.79709816e-01, -1.24639511e-01,
       -2.68115044e-01, -2.98548698e-01, -3.67387772e-01, -1.54348373e-01,
       -2.81884193e-01, -6.10142708e-01, -4.17393684e-01, -1.24639511e-01,
       -8.94924164e-01, -3.86957169e-01, -2.72462845e-01, -1.81159973e-02,
       -1.26085281e-01, -3.59418869e-01, -3.19562912e-01, -2.92027473e-01,
       -2.31885910e-02, -2.95651436e-01, -5.72462082e-01, -3.44926834e-01,
       -4.20289040e-01, -7.94928551e-01,  4.84779358e-01, -1.96374893e-01,
       -1.81159973e-01,  2.42025375e-01,  3.32607269e-01,  2.65216827e-01,
        4.10145760e-01,  2.68840790e-01,  2.68840790e-01,  2.68840790e-01,
        1.76811218e-01,  3.22463989e-01,  0.00000000e+00,  4.63800430e-02,
       -1.86957359e-01, -4.20303345e-02,  1.74638748e-01,  3.10869217e-01,
        7.53622055e-02,  1.07970238e-01,  1.40577316e-01,  2.94930458e-01,
        4.93479729e-01,  1.68113708e-01,  1.60869598e-01,  7.53622055e-02,
       -6.52027130e-03,  3.41305733e-01,  1.60869598e-01,  7.60898590e-02,
       -2.76088715e-01, -1.87680244e-01], dtype=float32)
Coordinates:
  * region        (region) object 'b8b6c57e-6fe4-432c-9130-f4d6e4598d19' ... ...
    constituency  (region) <U9 'E14001063' 'E14001064' ... 'W07000112'

In [61]:
region_da

<xarray.DataArray 'cdd_anom' (region: 650)>
array([ 2.42757797e-01, -1.60142899e-01,             nan,  7.24601746e-02,
        3.44408572e-01, -1.89130783e-01,  5.37681580e-01,             nan,
        3.39855194e-01,  5.15073776e-01,             nan, -4.42037582e-02,
                   nan, -3.29166293e-01,             nan,             nan,
       -2.49636412e-01,             nan,             nan,  3.40580940e-01,
                   nan,             nan,             nan,             nan,
       -2.41575241e-01,  4.09903198e-01,             nan,  4.48259741e-01,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
        4.41970825e-02,             nan, -2.78551489e-01,             nan,
                   nan, -2.81162262e-01, -3.60873222e-01, -5.14491081e-01,
       -4.25360680e-01,  2.60868073e-01, -6.73904419e-02,             nan,
       -4.09420967e-01,             nan,             nan,  2.78501511e-01,
                   nan,             nan,             nan,             nan,
                   nan,  9.42058563e-02,  5.82028568e-01,             nan,
        3.29704285e-01,             nan,  3.27175140e-01,  8.18475723e-01,
       -4.57246631e-01, -3.38587284e-01,  3.24640274e-01,             nan,
                   nan,  5.61592102e-01,             nan,  4.30433273e-01,
                   nan,  3.36956263e-01,             nan, -7.24601746e-02,
        2.42029190e-01,             nan,  3.24276924e-01, -2.31157303e-01,
...
        1.23198824e-02, -1.84124798e-01, -3.06519508e-01, -3.36746752e-01,
                   nan, -2.38405704e-01, -2.38043785e-01, -5.84980920e-02,
       -2.46506840e-01,             nan, -4.00905371e-01, -3.36474091e-01,
       -4.35505867e-01,             nan, -5.77535629e-01,             nan,
       -3.63766670e-01, -2.77535439e-01,             nan,             nan,
                   nan, -2.79709816e-01,             nan,             nan,
       -2.41666317e-01, -3.80929857e-01, -3.94563198e-01, -1.54348373e-01,
       -2.43100882e-01, -5.08115768e-01, -2.90684700e-01, -1.98189259e-01,
       -5.54891109e-01,             nan, -1.52445078e-01,             nan,
       -1.38645813e-01, -2.99847215e-01, -3.03621769e-01, -2.49891952e-01,
       -7.89861679e-02, -2.33930200e-01, -4.02608752e-01, -3.42391014e-01,
       -3.23043823e-01, -4.27493155e-01,  4.84779358e-01, -2.63404369e-01,
       -2.64822453e-01,  4.71014023e-01,  3.03653151e-01,             nan,
        2.62318432e-01,  2.68840790e-01,             nan,             nan,
        1.76811218e-01,             nan,  1.53726533e-01, -9.72210541e-02,
       -2.01450825e-01, -7.29469061e-02,  2.26449251e-01,  2.19747305e-01,
        1.57245323e-01,  2.75602192e-01,  2.31399953e-01,  2.07246527e-01,
        4.24274921e-01,  1.68113708e-01,  3.89130592e-01,  3.22463989e-01,
        1.93238258e-01,             nan,  1.60869598e-01,  1.00002289e-01,
       -6.49763718e-02, -3.05072933e-01], dtype=float32)
Coordinates:
  * region   (region) int64 0 1 2 3 4 5 6 7 ... 642 643 644 645 646 647 648 649

In [58]:
region_all

<xarray.DataArray 'cdd_anom' (region: 650)>
array([ 2.42757797e-01, -1.60142899e-01,  0.00000000e+00,  7.24601746e-02,
        3.44408572e-01, -1.89130783e-01,  5.37681580e-01,  0.00000000e+00,
        3.39855194e-01,  5.15073776e-01,  0.00000000e+00, -4.42037582e-02,
        0.00000000e+00, -3.29166293e-01,  0.00000000e+00,  0.00000000e+00,
       -2.49636412e-01,  0.00000000e+00,  0.00000000e+00,  3.40580940e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -2.41575241e-01,  4.09903198e-01,  0.00000000e+00,  4.48259741e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        4.41970825e-02,  0.00000000e+00, -2.78551489e-01,  0.00000000e+00,
        0.00000000e+00, -2.81162262e-01, -3.60873222e-01, -5.14491081e-01,
       -4.25360680e-01,  2.60868073e-01, -6.73904419e-02,  0.00000000e+00,
       -4.09420967e-01,  0.00000000e+00,  0.00000000e+00,  2.78501511e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  9.42058563e-02,  5.82028568e-01,  0.00000000e+00,
        3.29704285e-01,  0.00000000e+00,  3.27175140e-01,  8.18475723e-01,
       -4.57246631e-01, -3.38587284e-01,  3.24640274e-01,  0.00000000e+00,
        0.00000000e+00,  5.61592102e-01,  0.00000000e+00,  4.30433273e-01,
        0.00000000e+00,  3.36956263e-01,  0.00000000e+00, -7.24601746e-02,
        2.42029190e-01,  0.00000000e+00,  3.24276924e-01, -2.31157303e-01,
...
        1.23198824e-02, -1.84124798e-01, -3.06519508e-01, -3.36746752e-01,
        0.00000000e+00, -2.38405704e-01, -2.38043785e-01, -5.84980920e-02,
       -2.46506840e-01,  0.00000000e+00, -4.00905371e-01, -3.36474091e-01,
       -4.35505867e-01,  0.00000000e+00, -5.77535629e-01,  0.00000000e+00,
       -3.63766670e-01, -2.77535439e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.79709816e-01,  0.00000000e+00,  0.00000000e+00,
       -2.41666317e-01, -3.80929857e-01, -3.94563198e-01, -1.54348373e-01,
       -2.43100882e-01, -5.08115768e-01, -2.90684700e-01, -1.98189259e-01,
       -5.54891109e-01,  0.00000000e+00, -1.52445078e-01,  0.00000000e+00,
       -1.38645813e-01, -2.99847215e-01, -3.03621769e-01, -2.49891952e-01,
       -7.89861679e-02, -2.33930200e-01, -4.02608752e-01, -3.42391014e-01,
       -3.23043823e-01, -4.27493155e-01,  4.84779358e-01, -2.63404369e-01,
       -2.64822453e-01,  4.71014023e-01,  3.03653151e-01,  0.00000000e+00,
        2.62318432e-01,  2.68840790e-01,  0.00000000e+00,  0.00000000e+00,
        1.76811218e-01,  0.00000000e+00,  1.53726533e-01, -9.72210541e-02,
       -2.01450825e-01, -7.29469061e-02,  2.26449251e-01,  2.19747305e-01,
        1.57245323e-01,  2.75602192e-01,  2.31399953e-01,  2.07246527e-01,
        4.24274921e-01,  1.68113708e-01,  3.89130592e-01,  3.22463989e-01,
        1.93238258e-01,  0.00000000e+00,  1.60869598e-01,  1.00002289e-01,
       -6.49763718e-02, -3.05072933e-01])
Coordinates:
    constituency  (region) float64 nan nan nan nan nan ... nan nan nan nan nan
  * region        (region) object 0 1 2 3 4 5 6 ... 643 644 645 646 647 648 649
    name          (region) object 'E14001063' 'E14001064' ... 'W07000112'

<xarray.DataArray 'cdd_anom' (region: 0)>
array([], dtype=float32)
Coordinates:
  * region        (region) object 
    constituency  (region) <U9